In [1]:
import pandas as pd 
import time
import numpy as np

file_name = 'today_province'+'_'+time.strftime('%Y_%m_%d',time.localtime(time.time()))+'.csv'
china_data = pd.read_csv(file_name)

print(china_data.keys())


Index(['id', 'lastUpdateTime', 'name', 'total_confirm', 'total_suspect',
       'total_heal', 'total_dead', 'total_severe', 'total_input',
       'total_newConfirm', 'total_newDead', 'total_newHeal', 'today_confirm',
       'today_suspect', 'today_heal', 'today_dead', 'today_severe',
       'today_storeConfirm'],
      dtype='object')


In [2]:
import pyecharts
# 调整配置项
import pyecharts.options as opts
# Map类用于绘制地图
from pyecharts.charts import Map,Grid,Page
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts


In [3]:
china_data['existing_confirm'] = china_data['total_confirm'] - china_data['total_heal'] - china_data['total_dead']

china_data.head()#dataFrame有head()函数

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal,today_confirm,today_suspect,today_heal,today_dead,today_severe,today_storeConfirm,existing_confirm
0,420000,2021-06-04 00:01:23,湖北,68159,0,63646,4512,0,0,NaN,NaN,NaN,0,NaN,0,0,NaN,0,1
1,810000,2021-06-04 00:01:23,香港,11849,0,11576,210,0,0,NaN,NaN,NaN,0,NaN,1,0,NaN,0,63
2,700000,2021-06-04 00:01:22,台湾,9974,0,1133,166,0,0,NaN,NaN,NaN,1132,NaN,0,29,NaN,1103,8675
3,440000,2021-06-04 00:01:23,广东,2498,0,2388,8,0,0,NaN,NaN,NaN,17,NaN,0,0,NaN,17,102
4,310000,2021-06-04 00:01:22,上海,2111,0,2033,7,0,0,NaN,NaN,NaN,3,NaN,2,0,NaN,1,71


In [4]:
china_today_existingconfirm = china_data['existing_confirm'].sum()
china_today_confirm = china_data['today_confirm'].sum()
china_totalconfirm = china_data['total_confirm'].sum()
china_totalheal = china_data['total_heal'].sum()
china_totaldead = china_data['total_dead'].sum()

table3 = Table()
headers = ["现存确诊","累计确诊","今日新增","累计治愈","累计死亡"]
rows=[[china_today_existingconfirm,china_totalconfirm,china_today_confirm,china_totalheal,china_totaldead]]#rows是序列类型，[[]]

table3.add(headers,rows)

In [5]:
map_data = china_data[['name','existing_confirm']].values.tolist()
map_data[:5]

[['湖北', 1], ['香港', 63], ['台湾', 8675], ['广东', 102], ['上海', 71]]

In [6]:
table1_data = china_data[['name','existing_confirm','total_confirm','today_confirm','total_heal','total_dead']].values.tolist()#values不可缺少
table1_data.sort(key = lambda x:x[1],reverse = True)
table1_data = table1_data[:10]

In [7]:
table1 = Table()
headers = ["省份","现存确诊","累计确诊","今日新增","累计治愈","累计死亡"]
rows = []
for data in table1_data:
    rows.append(data)
table1.add(headers,rows)

#list1 = map_data[1],此时的map_data[1]返回的时第二个数据，而不是第二列所有元素
maxnum = 0
for data in map_data:
    if data[1]>maxnum:
        maxnum = data[1]
print(maxnum)

8675


In [8]:
map1= Map().add(series_name = "现存确诊人数", # 设置提示框标签
                 data_pair = map_data, # 输入数据
                 maptype = "china", # 设置地图类型为世界地图
                 is_map_symbol_show = False # 不显示标记点
                ) 

map1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示国家（标签）名称 
map1.set_global_opts(title_opts = opts.TitleOpts(title=" 中国现存确诊人数地图"), # 设置图标题
                     # 设置视觉映射配置项
                     visualmap_opts = opts.VisualMapOpts(pieces=[ # 自定义分组的分点和颜色
                                                               {"min": 1000,"color":"#800000"}, # 栗色
                                                               {"min": 100, "max": 1000, "color":"#B22222"}, # 耐火砖
                                                               {"min": 11, "max": 99,"color":"#CD5C5C"}, # 印度红
                                                               {"min": 1, "max": 10, "color":"#BC8F8F"}, # 玫瑰棕色
                                                               {"max": 1, "color":"#FFE4E1"}, # 薄雾玫瑰
                                                              ],
                     is_piecewise = True
                         #is_calculable=True,
                #dimension=0,
                #range_text=["High", "Low"],
                #range_color=["lightskyblue", "yellow", "orangered"],
                #textstyle_opts=opts.TextStyleOpts(color="#ddd"),
                #min_=0,
                #max_=maxnum,
                                                        ))  # 显示分段式图例


In [9]:
map2_data = china_data[['name','total_confirm']].values.tolist()
map2_data[:10]

[['湖北', 68159],
 ['香港', 11849],
 ['台湾', 9974],
 ['广东', 2498],
 ['上海', 2111],
 ['黑龙江', 1612],
 ['浙江', 1364],
 ['河北', 1317],
 ['河南', 1315],
 ['北京', 1061]]

In [10]:
map2 = Map().add(series_name = "累计确诊人数", # 设置提示框标签
                 data_pair = map2_data, # 输入数据
                 maptype = "china", # 设置地图类型为世界地图
                 is_map_symbol_show = False # 不显示标记点
                ) 

map2.set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示国家（标签）名称 
map2.set_global_opts(title_opts = opts.TitleOpts(title=" 中国累计确诊人数地图"), # 设置图标题
                     # 设置视觉映射配置项
                     visualmap_opts = opts.VisualMapOpts(pieces=[ # 自定义分组的分点和颜色
                                                               {"min": 10000,"color":"#800000"}, # 栗色
                                                               {"min": 5000, "max": 9999, "color":"#B22222"}, # 耐火砖
                                                               {"min": 999, "max": 4999,"color":"#CD5C5C"}, # 印度红
                                                               {"min": 100, "max": 999, "color":"#BC8F8F"}, # 玫瑰棕色
                                                               {"max": 99, "color":"#FFE4E1"}, # 薄雾玫瑰
                                                              ],
                     is_piecewise = True))  # 显示分段式图例


In [11]:
table2_data = china_data[['name','total_confirm','existing_confirm','today_confirm','total_heal','total_dead']].values.tolist()#values不可缺少
table2_data.sort(key = lambda x:x[1],reverse = True)
table2_data = table2_data[:10]

In [12]:
table2 = Table()
headers = ["省份","累计确诊","现存确诊","今日新增","累计治愈","累计死亡"]
rows = []
for data in table2_data:
    rows.append(data)
table2.add(headers,rows)

In [13]:
page = Page(layout=Page.SimplePageLayout)
page.add(
    map1,
    table3,
    table1,
    map2,
    table2,
)
page.render("中国疫情地图.html")

'C:\\Users\\admin\\Desktop\\jupyter notebook\\中国疫情地图.html'